<a href="https://colab.research.google.com/github/Stuna-alice/Stuna-TestGO/blob/Python_version_2_automate/C_backend.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install onnx
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)



from google.colab import drive
import os
import zipfile
import torch
import sys
from googleapiclient.discovery import build
from googleapiclient.http import MediaIoBaseDownload
from io import BytesIO

# Set the working directory
working_directory = '/content/gdrive/MyDrive/Winforms_data/yolov5'

os.chdir(working_directory)

# Function to read parameters from the text file
def read_parameters(file_path):
    parameters = {}
    with open(file_path, 'r') as file:
        lines = file.readlines()
        for line in lines:
            # Check if the line contains a colon
            if ':' in line:
                key, value = line.strip().split(':', 1)
                parameters[key.strip()] = value.strip()
    return parameters

# Specify the path to the zipfiles folder
zipfiles_folder_path = os.path.join(working_directory, 'zipfiles')

# List files in the zipfiles folder
zip_files = [f for f in os.listdir(zipfiles_folder_path) if f.endswith('.zip')]

# Find the latest uploaded zip file
latest_zip_file = None
latest_zip_time = 0

for zip_file in zip_files:
    zip_file_path = os.path.join(zipfiles_folder_path, zip_file)
    zip_file_time = os.path.getctime(zip_file_path)
    if zip_file_time > latest_zip_time:
        latest_zip_time = zip_file_time
        latest_zip_file = zip_file

if latest_zip_file is not None:
    # Download the latest zip file
    zip_file_name = latest_zip_file
    zip_file_path = os.path.join(zipfiles_folder_path, zip_file_name)

    # Extract the contents of the zip file
    with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
        zip_ref.extractall(working_directory)

    print(f"'{zip_file_name}' extracted to '{working_directory}'.")

    # Read user parameters from the text file
    base_name = os.path.splitext(os.path.basename(zip_file_name))[0]
    text_file_path = os.path.join(zipfiles_folder_path, f'{base_name}.txt')
    if not os.path.exists(text_file_path):
        print(f"Error: Text file '{base_name}.txt' not found.")
    else:
        parameters = read_parameters(text_file_path)

        # Print all parameters
        print("All Parameters:")
        for key, value in parameters.items():
            print(f"{key}: {value}")

        # Check if required parameters are present
        required_params = ['img', 'batch', 'epochs', 'weights']
        missing_params = [param for param in required_params if param not in parameters]

        if missing_params:
            print(f"Error: Missing parameters in text file: {', '.join(missing_params)}")
        else:
            # Train YOLOv5s with user parameters
            weights_path = parameters['weights']
            !python train.py --img {parameters['img']} --batch {parameters['batch']} --epochs {parameters['epochs']} --data {yaml_file_path} --weights {weights_path} --cache

            sys.path.append('/content/gdrive/MyDrive/Winforms_data/yolov5/models')
            # Load the YOLOv5 model from yolov5s.yaml
            model_config_path = '/content/gdrive/MyDrive/Winforms_data/yolov5/models/yolov5s.yaml'
            from models.yolo import Model
            model = Model(model_config_path)

            model_name = f"{base_name}.pth"
            weights_dir = os.path.join(working_directory, 'weights')

            # Create the 'weights' directory if it doesn't exist
            os.makedirs(weights_dir, exist_ok=True)

            model_path = os.path.join(weights_dir, model_name)
            torch.save(model.state_dict(), model_path)
            # Convert the .pth file to ONNX format
            onnx_path = f'{working_directory}/zipfiles/{base_name}.onnx'
            dummy_input = torch.randn(int(parameters['batch']), 3, int(parameters['img']), int(parameters['img']))
            torch.onnx.export(model, dummy_input, onnx_path)

            print(f"Model saved as '{model_name}' and exported to '{onnx_path}'.")
else:
    print("No zip files found in the specified folder.")


In [ ]:


#BackUp
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

import os
import zipfile
import torch
from models.experimental import attempt_load
from utils.general import check_img_size

# Set the working directory
working_directory = '/content/gdrive/MyDrive/Winforms_data/yolov5'
os.chdir(working_directory)

# Function to read parameters from the text file
def read_parameters(file_path):
    parameters = {}
    with open(file_path, 'r') as file:
        lines = file.readlines()
        for line in lines:
            key, value = line.strip().split(':')
            parameters[key.strip()] = value.strip()
    return parameters

# Define the zip file name
zip_file_name = 'chickenDB.zip'

# Extract the base name of the zip file (excluding the extension)
base_name = os.path.splitext(os.path.basename(zip_file_name))[0]

# Update the YAML file with the extracted base name
yaml_content = f"""
train: {working_directory}/dataset_extract/{base_name}/images/train/
val: {working_directory}/dataset_extract/{base_name}/images/val/
nc: 1
names: ['{base_name}']
"""

# Write the updated YAML content to a file
yaml_file_path = f'{working_directory}/data/defaultYamlpath.yaml'
with open(yaml_file_path, 'w') as yaml_file:
    yaml_file.write(yaml_content)

# Create the destination folder if it doesn't exist
extracted_folder_path = f'{working_directory}/dataset_extract'
os.makedirs(extracted_folder_path, exist_ok=True)

# Define the path to the zip file and the destination folder
zip_file_path = f'{working_directory}/zipfiles/{zip_file_name}'

# Extract the contents of the zip file
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(extracted_folder_path)

print(f" '{zip_file_name}' extracted to '{extracted_folder_path}'.")

# Read user parameters from the text file
text_file_path = f'{working_directory}/zipfiles/{base_name}.txt'
if not os.path.exists(text_file_path):
    print(f"Error: Text file '{base_name}.txt' not found.")
else:
    parameters = read_parameters(text_file_path)

    # Check if required parameters are present
    required_params = ['img', 'batch', 'epochs', 'weights']
    missing_params = [param for param in required_params if param not in parameters]

    if missing_params:
        print(f"Error: Missing parameters in text file: {', '.join(missing_params)}")
    else:
        # Train YOLOv5 with user parameters
        !python train.py --img {parameters['img']} --batch {parameters['batch']} --epochs {parameters['epochs']} --data {yaml_file_path} --weights {parameters['weights']} --cache

        # Load the trained YOLOv5 model
        model = attempt_load(parameters['weights'], map_location=torch.device('cuda' if torch.cuda.is_available() else 'cpu'))
        img_size = check_img_size(640, s=model.stride.max())  # Change 640 to your desired image size

        # Export the model to ONNX
        onnx_path = f'{working_directory}/yolov5.onnx'
        model.model[-1].export = torch.jit.ExportType.ONNX
        model.model[-1].optimize_for_inference(img_size)
        model.model[-1].export.export_path = onnx_path
        model.model[-1].export.image_size = img_size
        model.model[-1].export.single_image = True
        model.model[-1].forward = model.model[-1].export.forward
        model.model[-1].export_module(torch.rand(1, 3, img_size, img_size).to(model.model[-1].device), verbose=True)

        print(f"Model exported to ONNX: {onnx_path}")


ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Mounted at /content/gdrive
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/IPython/core/interactiveshell.py", line 3553, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-6-fff778ee7f65>", line 8, in <cell line: 8>
    from models.experimental import attempt_load
  File "<frozen importlib._bootstrap>", line 1027, in _find_and_load
  File "<frozen importlib._bootstrap>", line 1002, in _find_and_load_unlocked
  File "<frozen importlib._bootstrap>", line 945, in _find_spec
  File "<frozen importlib._bootstrap_external>", line 1439, in find_spec
  File "<frozen importlib._bootstrap_external>", line 1408, in _get_spec
  File "<frozen importlib._bootstrap_external>", line 1366, in _path_importer_cache
OSError: [Errno 107] Transport endpoint is not connected

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/IPython/cor

In [ ]:

#running
from google.colab import drive
import os
import zipfile
import subprocess

# Mount Google Drive
drive.mount('/content/gdrive', force_remount=True)

# Set the working directory
working_directory = '/content/gdrive/MyDrive/Winforms_data/yolov5'
os.chdir(working_directory)

# Function to read parameters from the text file
def read_parameters(file_path):
    parameters = {}
    with open(file_path, 'r') as file:
        lines = file.readlines()
        for line in lines:
            # Check if the line contains a colon
            if ':' in line:
                key, value = line.strip().split(':', 1)
                parameters[key.strip()] = value.strip()
    return parameters

# Define the zip file name
zip_file_name = 'chickenDB.zip'

# Extract the base name of the zip file (excluding the extension)
base_name = os.path.splitext(os.path.basename(zip_file_name))[0]

# Update the YAML file with the extracted base name
yaml_content = f"""
train: {working_directory}/dataset_extract/{base_name}/images/train/
val: {working_directory}/dataset_extract/{base_name}/images/val/
nc: 1
names: ['{base_name}']
"""

# Write the updated YAML content to a file
yaml_file_path = f'{working_directory}/data/defaultYamlpath.yaml'
with open(yaml_file_path, 'w') as yaml_file:
    yaml_file.write(yaml_content)

# Create the destination folder if it doesn't exist
extracted_folder_path = f'{working_directory}/dataset_extract'
os.makedirs(extracted_folder_path, exist_ok=True)

# Define the path to the zip file and the destination folder
zip_file_path = f'{working_directory}/zipfiles/{zip_file_name}'

# Extract the contents of the zip file
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(extracted_folder_path)

print(f"'{zip_file_name}' extracted to '{extracted_folder_path}'.")

# Read user parameters from the text file
text_file_path = f'{working_directory}/zipfiles/{base_name}.txt'
if not os.path.exists(text_file_path):
    print(f"Error: Text file '{base_name}.txt' not found.")
else:
    parameters = read_parameters(text_file_path)

    # Print all parameters
    print("All Parameters:")
    for key, value in parameters.items():
        print(f"{key}: {value}")

    # Check if required parameters are present
    required_params = ['img', 'batch', 'epochs', 'weights']
    missing_params = [param for param in required_params if param not in parameters]

    if missing_params:
        print(f"Error: Missing parameters in text file: {', '.join(missing_params)}")
    else:
        # Train YOLOv5s with user parameters
        weights_path = parameters['weights']
        training_cmd = "python train.py --img {img} --batch {batch} --epochs {epochs} --data {data} --weights {weights} --cache".format(
            img=parameters['img'],
            batch=parameters['batch'],
            epochs=parameters['epochs'],
            data=yaml_file_path,
            weights=weights_path
        )
        print(f"Training command: {training_cmd}")

        # Execute training command using subprocess.run
        subprocess.run(training_cmd, shell=True)

        # Export the trained model to ONNX format
        output_onnx_path = f'{working_directory}/zipfiles/{base_name}.onnx'
        onnx_export_cmd = "python export.py --weights {weights} --img-size {img_size} --include pb --dynamic --simplify --optimize 0 --simplify-num-ends 0 --simplify-num-mid 0 --simplify-num-start 0 --simplify-threshold 0.5 --simplify-method 0 --simplify-img-size {img_size} --simplify-nc 1 --dynamic-nc --simplify-preserve-reduction --output {output_path}".format(
            weights=weights_path,
            img_size=parameters['img'],
            output_path=output_onnx_path
        )

        print(f"Exporting model to ONNX. Command: {onnx_export_cmd}")

        try:
            # Execute ONNX export command using subprocess.run
            subprocess.run(onnx_export_cmd, shell=True)
            print(f"Export successful.")
        except Exception as e:
            print(f"Error during ONNX export: {e}")


In [ ]:
from google.colab import drive
import os
import zipfile
import subprocess
# Mount Google Drive
drive.mount('/content/gdrive', force_remount=True)

# Set the working directory
working_directory = '/content/gdrive/MyDrive/Winforms_data/yolov5'
os.chdir(working_directory)

# Function to read parameters from the text file
def read_parameters(file_path):
    parameters = {}
    with open(file_path, 'r') as file:
        lines = file.readlines()
        for line in lines:
            # Check if the line contains a colon
            if ':' in line:
                key, value = line.strip().split(':', 1)
                parameters[key.strip()] = value.strip()
    return parameters

# Define the zip file name
zip_file_name = 'chickenDB.zip'

# Extract the base name of the zip file (excluding the extension)
base_name = os.path.splitext(os.path.basename(zip_file_name))[0]

# Update the YAML file with the extracted base name
yaml_content = f"""
train: {working_directory}/dataset_extract/{base_name}/images/train/
val: {working_directory}/dataset_extract/{base_name}/images/val/
nc: 1
names: ['{base_name}']
"""

# Write the updated YAML content to a file
yaml_file_path = f'{working_directory}/data/defaultYamlpath.yaml'
with open(yaml_file_path, 'w') as yaml_file:
    yaml_file.write(yaml_content)

# Create the destination folder if it doesn't exist
extracted_folder_path = f'{working_directory}/dataset_extract'
os.makedirs(extracted_folder_path, exist_ok=True)

# Define the path to the zip file and the destination folder
zip_file_path = f'{working_directory}/zipfiles/{zip_file_name}'

# Extract the contents of the zip file
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(extracted_folder_path)

print(f"'{zip_file_name}' extracted to '{extracted_folder_path}'.")

# Read user parameters from the text file
text_file_path = f'{working_directory}/zipfiles/{base_name}.txt'
if not os.path.exists(text_file_path):
    print(f"Error: Text file '{base_name}.txt' not found.")
else:
    parameters = read_parameters(text_file_path)

    # Print all parameters
    print("All Parameters:")
    for key, value in parameters.items():
        print(f"{key}: {value}")

    # Check if required parameters are present
    required_params = ['img', 'batch', 'epochs', 'weights']
    missing_params = [param for param in required_params if param not in parameters]

    if missing_params:
        print(f"Error: Missing parameters in text file: {', '.join(missing_params)}")
    else:
        # Train YOLOv5s with user parameters
        weights_path = parameters['weights']
        !python train.py --img {parameters['img']} --batch {parameters['batch']} --epochs {parameters['epochs']} --data {yaml_file_path} --weights {weights_path} --cache

        # Check if the export.py script exists
        export_script_path = os.path.join(working_directory, 'export.py')
        if not os.path.exists(export_script_path):
            print(f"Error: 'export.py' not found in the specified directory.")
        else:
            # Export the trained model to ONNX format
            output_onnx_path = f'{working_directory}/{base_name}.onnx'
            onnx_export_cmd = f"python {export_script_path} --weights {weights_path} --img-size {parameters['img']} --include pb --dynamic --simplify --optimize 0 --simplify-num-ends 0 --simplify-num-mid 0 --simplify-num-start 0 --simplify-threshold 0.5 --simplify-method 0 --simplify-img-size {parameters['img']} --simplify-nc 1 --dynamic-nc --simplify-preserve-reduction --output {output_onnx_path}"

            # Check if the export command was successful
            exit_code = os.system(onnx_export_cmd)

            # Capture the output of the command
            onnx_export_output = os.popen(onnx_export_cmd).read()

            if exit_code == 0:
                print(f"ONNX file exported to '{output_onnx_path}'.")
            else:
                print(f"Error: Export command exited with code {exit_code}.")
                print(f"Export command output:\n{onnx_export_output}")


In [ ]:

from google.colab import drive
import os
import zipfile
import shutil

# Mount Google Drive
drive.mount('/content/gdrive', force_remount=True)

# Set the working directory
working_directory = '/content/gdrive/MyDrive/Winforms_data/yolov5'
os.chdir(working_directory)

# Function to read parameters from the text file
def read_parameters(file_path):
    parameters = {}
    with open(file_path, 'r') as file:
        lines = file.readlines()
        for line in lines:
            key, value = line.strip().split(':')
            parameters[key.strip()] = value.strip()
    return parameters

# Define the zip file name
zip_file_name = 'chickenDB.zip'

# Extract the base name of the zip file (excluding the extension)
base_name = os.path.splitext(os.path.basename(zip_file_name))[0]

# Update the YAML file with the extracted base name
yaml_content = f"""
train: {working_directory}/dataset_extract/{base_name}/images/train/
val: {working_directory}/dataset_extract/{base_name}/images/val/
nc: 1
names: ['{base_name}']
"""

# Write the updated YAML content to a file
yaml_file_path = f'{working_directory}/data/defaultYamlpath.yaml'
with open(yaml_file_path, 'w') as yaml_file:
    yaml_file.write(yaml_content)

# Create the destination folder if it doesn't exist
extracted_folder_path = f'{working_directory}/dataset_extract'
os.makedirs(extracted_folder_path, exist_ok=True)

# Define the path to the zip file and the destination folder
zip_file_path = f'{working_directory}/zipfiles/{zip_file_name}'

# Extract the contents of the zip file
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(extracted_folder_path)

print(f" '{zip_file_name}' extracted to '{extracted_folder_path}'.")

# Read user parameters from the text file
text_file_path = f'{working_directory}/zipfiles/{base_name}.txt'
if not os.path.exists(text_file_path):
    print(f"Error: Text file '{base_name}.txt' not found.")
else:
    parameters = read_parameters(text_file_path)

    # Check if required parameters are present
    required_params = ['img', 'batch', 'epochs', 'weights']
    missing_params = [param for param in required_params if param not in parameters]

    if missing_params:
        print(f"Error: Missing parameters in text file: {', '.join(missing_params)}")
    else:
        # Train YOLOv5s with user parameters
        !python train.py --img {parameters['img']}  --batch {parameters['batch']} --epochs {parameters['epochs']} --data {yaml_file_path} --weights {parameters['weights']} --cache

        # Export the trained model to ONNX format
        onnx_file_name = f'{base_name}.onnx'
        output_onnx_path = f'{working_directory}/{onnx_file_name}'
        export_command = f"python export.py --weights runs/train/exp/weights/best.pt --img-size {parameters['img']} --batch-size {parameters['batch']} --include pb"  # Modify the export.py command based on your requirements

        !{export_command}

        # Move the exported ONNX file to the desired location
        shutil.move('yolov5s.onnx', output_onnx_path)

        print(f"Trained model exported to '{output_onnx_path}'.")

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

# Change directory to allow Google to access GDrive
%cd /content/gdrive/MyDrive/Winforms_data

# Clone YOLOv5 repository
!git clone https://github.com/ultralytics/yolov5

# Change directory to the YOLOv5 repository
%cd yolov5

# Install required packages
%pip install -qr requirements.txt comet_ml

import torch
import utils
display = utils.notebook_init()  # checks
import zipfile
import os


# Define the path to the zip file and the destination folder
zipFile_name = 'chickenDB.zip'
zip_file_path = '/content/gdrive/MyDrive/Winforms_data/yolov5/zipfiles/' + zipFile_name
extracted_folder_path = '/content/gdrive/MyDrive/Winforms_data/yolov5/dataset_extract'

# Create the destination folder if it doesn't exist
os.makedirs(extracted_folder_path, exist_ok=True)

# Extract the contents of the zip file
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(extracted_folder_path)

print(f" '{zipFile_name}' extracted to '{extracted_folder_path}'.")


# make label code
#training
# Train YOLOv5s on marmot for 3 epochs
!python train.py --img 640 --batch 20 --epochs 50 --data controlDB.yaml --weights yolov5s.pt --cache



In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

# Change directory to allow Google to access GDrive
%cd /content/gdrive/MyDrive/Winforms_data

# Clone YOLOv5 repository
!git clone https://github.com/ultralytics/yolov5

# Change directory to the YOLOv5 repository
%cd yolov5

# Install required packages
%pip install -qr requirements.txt comet_ml

import torch
import utils
display = utils.notebook_init()  # checks
import zipfile
import os
from pathlib import Path
# Define the zip file name
zip_file_name = 'chickenDB.zip'  # Change this to the actual zip file name

# Extract the base name of the zip file (excluding the extension)
base_name = os.path.splitext(os.path.basename(zip_file_name))[0]

# Update the YAML file with the extracted base name
yaml_content = f"""
train: /content/gdrive/MyDrive/Winforms_data/yolov5/dataset_extract/{base_name}/images/train/
val: /content/gdrive/MyDrive/Winforms_data/yolov5/dataset_extract/{base_name}/images/val/
nc: 1
names: ['{base_name}']
"""

# Write the updated YAML content to a file
yaml_file_path = '/content/gdrive/MyDrive/Winforms_data/yolov5/data/defaultYamlpath.yaml'
with open(yaml_file_path, 'w') as yaml_file:
    yaml_file.write(yaml_content)

# Create the destination folder if it doesn't exist
extracted_folder_path = f'/content/gdrive/MyDrive/Winforms_data/yolov5/dataset_extract'
os.makedirs(extracted_folder_path, exist_ok=True)

# Define the path to the zip file and the destination folder
zip_file_path = f'/content/gdrive/MyDrive/Winforms_data/yolov5/zipfiles/{zip_file_name}'

# Extract the contents of the zip file
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(extracted_folder_path)

print(f" '{zip_file_name}' extracted to '{extracted_folder_path}'.")

# Make label code
# training
# Train YOLOv5s on marmot for 3 epochs
!python train.py --img 640 --batch 21 --epochs 2 --data {yaml_file_path} --weights yolov5s.pt --cache

# After training, export the model to ONNX format
model_path = 'runs/train/exp/weights/best.pt'  # Change this path based on your training output
onnx_export_path = '/content/gdrive/MyDrive/Winforms_data/yolov5/onnx_exported_model.onnx'


# Export the trained model to ONNX format
!python export.py --img-size 640 --batch-size 1 --dynamic

# Find the ONNX file in the export directory
onnx_temp_path = Path('yolov5s.onnx')  # The name generated by YOLOv5 export script
onnx_temp_path.rename(onnx_export_path)

print(f"ONNX model exported to: {onnx_export_path}")

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

import os
import zipfile

# Set the working directory
working_directory = '/content/gdrive/MyDrive/Winforms_data/yolov5'
os.chdir(working_directory)

# Function to read parameters from the text file
def read_parameters(file_path):
    parameters = {}
    with open(file_path, 'r') as file:
        lines = file.readlines()
        for line in lines:
            key, value = line.strip().split(':')
            parameters[key.strip()] = value.strip()
    return parameters

# Define the zip file name
zip_file_name = 'chickenDB.zip'

# Extract the base name of the zip file (excluding the extension)
base_name = os.path.splitext(os.path.basename(zip_file_name))[0]

# Update the YAML file with the extracted base name
yaml_content = f"""
train: {working_directory}/dataset_extract/{base_name}/images/train/
val: {working_directory}/dataset_extract/{base_name}/images/val/
nc: 1
names: ['{base_name}']
"""

# Write the updated YAML content to a file
yaml_file_path = f'{working_directory}/data/defaultYamlpath.yaml'
with open(yaml_file_path, 'w') as yaml_file:
    yaml_file.write(yaml_content)

# Create the destination folder if it doesn't exist
extracted_folder_path = f'{working_directory}/dataset_extract'
os.makedirs(extracted_folder_path, exist_ok=True)

# Define the path to the zip file and the destination folder
zip_file_path = f'{working_directory}/zipfiles/{zip_file_name}'

# Extract the contents of the zip file
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(extracted_folder_path)

print(f" '{zip_file_name}' extracted to '{extracted_folder_path}'.")

# Read user parameters from the text file
text_file_path = f'{working_directory}/zipfiles/{base_name}.txt'
if not os.path.exists(text_file_path):
    print(f"Error: Text file '{base_name}.txt' not found.")
else:
    parameters = read_parameters(text_file_path)

    # Check if required parameters are present
    required_params = ['img', 'batch', 'epochs', 'weights']
    missing_params = [param for param in required_params if param not in parameters]

    if missing_params:
        print(f"Error: Missing parameters in text file: {', '.join(missing_params)}")
    else:

        # Train YOLOv5s with user parameters
        #train_command = f"!python train.py --img {parameters['img']} --batch {parameters['batch']} --epochs {parameters['epochs']} \n --data {yaml_file_path} --weights {parameters['weights']} --cache \n"

        !python train.py --img {parameters['img']}  --batch {parameters['batch']} --epochs {parameters['epochs']} --data {yaml_file_path} --weights {parameters['weights']} --cache


